主要代码

In [ ]:
import os
import json
from zhipuai import ZhipuAI
import re
from tqdm import tqdm  


def extract_text_from_file(file_path):
    """从文件中提取文本内容，包括分页信息"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    pages = re.split(r'(第 \d +页|\n\s*Page \d+\s*\n)', content)
    structured_pages = []
    
    for i in range(1, len(pages), 2):
        page_marker = pages[i].strip()
        page_content = pages[i+1].strip() if i+1 < len(pages) else ""
        if page_content:  
            structured_pages.append({
                "page_marker": page_marker,
                "content": page_content
            })
    
    return structured_pages

def generate_qa_pairs(client, text_segment, doc_name, page_info, few_shot_examples):
    """使用GLM-4模型生成QA对"""
    prompt = f"""请严格按以下要求生成QA对：
1. 问题必须针对文本中的具体条款
2. 答案必须直接引用原文片段
3. 使用此JSON格式（严禁嵌套其他结构）：
{{
  "question": "明确的问题",
  "answer": "原文中的完整答案",
  "content": "引用的原文（不超过100字）",
  "source_file": "{doc_name}",
  "page_num": "{page_info}"
}}

待分析文本：
{text_segment}
"""
    
    try:
        response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": "你是一个专业的文档分析助手，擅长从文本中提取关键信息生成问答对。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=1000
        )

        response_content = response.choices[0].message.content
        if response_content.startswith("```json"):
            response_content = response_content[7:-3].strip()  # 去除Markdown代码块
        
        qa_pair = json.loads(response_content)
        qa_pair.update({
            "source_file": doc_name,
            "page_num": page_info,
            "content": text_segment[:500] + "..." if len(text_segment) > 500 else text_segment  # 截断过长的文本
        })
        return qa_pair
    except Exception as e:
        print(f"\n生成QA对时出错: {e}")
        return None

def split_text_into_segments(text, max_length=600):
    """将文本分割成有意义的片段"""
    # 先尝试按段落分割
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    segments = []
    for para in paragraphs:
        if len(para) <= max_length:
            segments.append(para)
        else:
            # 长段落再按句子分割
            sentences = re.split(r'(?<=[。！？；;])\s+', para)
            current_segment = ""
            for sent in sentences:
                if len(current_segment) + len(sent) <= max_length:
                    current_segment += sent + " "
                else:
                    if current_segment:
                        segments.append(current_segment.strip())
                    current_segment = sent + " "
            if current_segment:
                segments.append(current_segment.strip())
    
    return segments

def process_documents(input_folder, output_path, few_shot_examples, qa_per_doc=5, min_text_length=100):
    """处理文件夹中的所有文档
    :param qa_per_doc: 每个文档最多生成的QA对数量
    :param min_text_length: 生成QA的最小文本长度(字符数)
    """
    client = ZhipuAI(api_key="2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES")  
    
    all_qa_pairs = []
    
    # 获取所有待处理文件
    file_list = [f for f in os.listdir(input_folder) if f.endswith(('.md', '.txt', '.pdf'))]
    total_files = len(file_list)
    
    print(f"\n开始处理，共发现 {total_files} 个文档")
    
    with tqdm(file_list, desc="处理进度", unit="文件") as pbar:
        for filename in pbar:
            pbar.set_postfix_str(f"当前文件: {filename[:15]}...")
            try:
                file_path = os.path.join(input_folder, filename)
                pages = extract_text_from_file(file_path)
                doc_qa_count = 0
                
                for page in pages:
                    segments = split_text_into_segments(page['content'])
                    for segment in segments:
                        if doc_qa_count >= qa_per_doc:
                            break
                        if len(segment) >= min_text_length:
                            qa_pair = generate_qa_pairs(
                                client, segment, filename, 
                                page['page_marker'], few_shot_examples
                            )
                            if qa_pair:  # 只有生成成功才计数
                                all_qa_pairs.append(qa_pair)
                                doc_qa_count += 1
                                pbar.set_postfix_str(f"当前文件: {filename[:15]}..., 已生成: {doc_qa_count}/{qa_per_doc}")
                
                pbar.set_postfix_str(f"完成: {filename[:15]}..., 生成 {doc_qa_count} QA对")
            except Exception as e:
                print(f"\n处理文件 {filename} 时出错: {e}")
                continue
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2, 
                 separators=(',', ': '), sort_keys=True)
    
    print(f"\n处理完成！共处理 {total_files} 个文档，生成 {len(all_qa_pairs)} 个QA对")
    print(f"结果已保存到: {os.path.abspath(output_path)}")

# 示例few-shot prompt
FEW_SHOT_EXAMPLES = """
{
    "examples": [
        {
            "question": "投资相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）首年寬限期有多久？若寬限期過後仍未繳交保費會怎樣？",
            "answer": "靈活投資寶╱萬利保障計劃╱宏寶保單將享有 30 日的寬限期，宏利精選投資保之寬限期為 31 日。若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。",
            "source_file": "电子行政运作手册_保单行政_2023_10.md",
            "page_num": "6",
            "content": "投資相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）\n\na) 寬限期\n\n假如首年內的逾期最低保費（MPRE/Contract Premium）或基金帳戶淨值不足以支付保單月費，客戶會收到保費備忘書。  \n靈活投資寶╱萬利保障計劃╱宏寶保單將享有30 日的寬限期（宏利精選投資保之寬限期為31 日）。  \n若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。"
        },
        {
            "question": "投資相連壽險計劃保單（邁駿投資理財計劃）在第二年淨保費測試（第十三、十四、十五個月結日），什麼情況下保險顧問的營業積分將被扣回？",
            "answer": "在第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 ≤（N - 2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。",
            "source_file": "电子行政运作手册_保单行政_2023_10.md",
            "page_num": "7",
            "content": "淨保費測試（Net Premium Test）：\n\n淨保費測試決定自保單生效日以後的淨已繳保費是否足夠支付到期定期總保費。  \n淨已繳保費 $\\mathbf{\\Sigma}=\\mathbf{\\Sigma}$ 已繳至基本儲蓄帳戶之保費總額（包括「自動繳付保費」之金額，如適用）–每次從基本儲蓄帳戶提取時 (如適用)超出當時免贖回費之基金提取限額的金額之累積總額保費備忘書（如自動轉帳拒付，則發出自動轉帳拒付通知書）將在（N）月結日的淨已繳保費 $\\leqq$ （N–1）月結日的到期定期總保費時發出。  \n另一份保費備忘書將於其後的一個月結日仍未收到所需的付款時發出。若連續三個月結日仍未收到所需的付款╱所需的付款不足以支付到期定期總保費及保單未能執行「自動繳付保費」時，保單月費中的欠繳保費手續費將從基本儲蓄帳戶內扣除。  \n於第一年淨保費測試（第十二、十三、十四個月結日）及第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 $\\leqq$ （N–2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。"
        }
    ]
}
"""

if __name__ == "__main__":
    input_folder = input("请输入文件夹路径: ").strip().strip('"').strip("'")
    output_path = input("请输入输出文件路径: ").strip().strip('"').strip("'")
    qa_per_doc = int(input("请输入每个文档最多生成的QA对数量(默认2): ") or 2)
    
    if not os.path.isdir(input_folder):
        print(f"错误: 文件夹路径不存在 - {input_folder}")
        exit(1)
    
    process_documents(
        input_folder=input_folder,
        output_path=output_path,
        few_shot_examples=FEW_SHOT_EXAMPLES,
        qa_per_doc=qa_per_doc
    )


开始处理，共发现 7 个文档


处理进度:   0%|          | 0/7 [00:00<?, ?文件/s, 当前文件: 守护无间危疾保 保单条款_20...]


生成QA对时出错: Extra data: line 9 column 1 (char 204)


处理进度:   0%|          | 0/7 [00:27<?, ?文件/s, 当前文件: 守护无间危疾保 保单条款_20..., 已生成: 1/2]


生成QA对时出错: Extra data: line 9 column 1 (char 435)

生成QA对时出错: Extra data: line 9 column 1 (char 245)

生成QA对时出错: Extra data: line 9 column 1 (char 286)

生成QA对时出错: Extra data: line 9 column 1 (char 287)


...


生成QA对时出错: Extra data: line 9 column 1 (char 174)

生成QA对时出错: Extra data: line 9 column 1 (char 201)


处理进度:  57%|█████▋    | 4/7 [05:20<03:28, 69.53s/文件, 当前文件: 活耀人生危疾保-产品手册.md..., 已生成: 1/2]


生成QA对时出错: Extra data: line 7 column 2 (char 201)

生成QA对时出错: Extra data: line 7 column 2 (char 213)

生成QA对时出错: Extra data: line 9 column 1 (char 307)

生成QA对时出错: Extra data: line 9 column 1 (char 201)


处理进度:  71%|███████▏  | 5/7 [07:04<02:41, 80.97s/文件, 当前文件: 电子行政运作手册_保单行政_2..., 已生成: 1/2]  


生成QA对时出错: Extra data: line 9 column 1 (char 180)

生成QA对时出错: Extra data: line 7 column 2 (char 231)

生成QA对时出错: Invalid \escape: line 4 column 28 (char 125)

生成QA对时出错: Extra data: line 9 column 1 (char 332)

生成QA对时出错: Extra data: line 9 column 1 (char 270)

生成QA对时出错: Extra data: line 7 column 2 (char 206)

生成QA对时出错: Extra data: line 7 column 2 (char 207)


尝试更改

In [ ]:
import os
import json
from zhipuai import ZhipuAI
import re
from tqdm import tqdm  


def extract_text_from_file(file_path):
    """从文件中提取文本内容，包括分页信息"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    pages = re.split(r'(第 \d +页|\n\s*Page \d+\s*\n)', content)
    structured_pages = []
    
    for i in range(1, len(pages), 2):
        page_marker = pages[i].strip()
        page_content = pages[i+1].strip() if i+1 < len(pages) else ""
        if page_content:  
            structured_pages.append({
                "page_marker": page_marker,
                "content": page_content
            })
    
    return structured_pages

def generate_qa_pairs(client, text_segment, doc_name, page_info, few_shot_examples):
    """使用GLM-4模型生成QA对"""
    prompt = f"""请严格按以下要求生成QA对：
1. 问题的生成来源要平均分布于整个文档内容
1. 问题必须针对文本中的具体内容
2. 答案必须来源于原文片段
3. 使用此JSON格式（严禁嵌套其他结构）：
{{
  "question": "明确的问题",
  "answer": "原文中的完整答案",
  "文本片段": "引用的原文（不超过100字）",
  "文档来源": "{doc_name}",
  "页码": "{page_info}"
}}

待分析文本：
{text_segment}
"""
    
    try:
        response = client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": "你是一个专业的文档分析助手，擅长从文本中提取关键信息生成问答对。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.2,
            max_tokens=1000
        )
        
        response_content = response.choices[0].message.content
        if response_content.startswith("```json"):
            response_content = response_content[7:-3].strip()  # 去除Markdown代码块
        
        qa_pair = json.loads(response_content)
        qa_pair.update({
            "文档来源": doc_name,
            "页码": page_info,
            "文本片段": text_segment[:500] + "..." if len(text_segment) > 500 else text_segment  # 截断过长的文本
        })
        return qa_pair
    except Exception as e:
        print(f"\n生成QA对时出错: {e}")
        return None

def split_text_into_segments(text, max_length=600):
    """将文本分割成有意义的片段"""
    # 先尝试按段落分割
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    
    segments = []
    for para in paragraphs:
        if len(para) <= max_length:
            segments.append(para)
        else:
            # 长段落再按句子分割
            sentences = re.split(r'(?<=[。！？；;])\s+', para)
            current_segment = ""
            for sent in sentences:
                if len(current_segment) + len(sent) <= max_length:
                    current_segment += sent + " "
                else:
                    if current_segment:
                        segments.append(current_segment.strip())
                    current_segment = sent + " "
            if current_segment:
                segments.append(current_segment.strip())
    
    return segments

def process_documents(input_folder, output_path, few_shot_examples, qa_per_doc=5, min_text_length=100):
    """处理文件夹中的所有文档
    :param qa_per_doc: 每个文档最多生成的QA对数量
    :param min_text_length: 生成QA的最小文本长度(字符数)
    """
    client = ZhipuAI(api_key="2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES")  
    
    all_qa_pairs = []
    
    # 获取所有待处理文件
    file_list = [f for f in os.listdir(input_folder) if f.endswith(('.md', '.txt', '.pdf'))]
    total_files = len(file_list)
    
    print(f"\n开始处理，共发现 {total_files} 个文档")
    
    # 使用tqdm创建进度条
    with tqdm(file_list, desc="处理进度", unit="文件") as pbar:
        for filename in pbar:
            pbar.set_postfix_str(f"当前文件: {filename[:15]}...")
            try:
                file_path = os.path.join(input_folder, filename)
                pages = extract_text_from_file(file_path)
                doc_qa_count = 0
                
                for page in pages:
                    segments = split_text_into_segments(page['content'])
                    for segment in segments:
                        
                        if doc_qa_count >= qa_per_doc:
                            break
                        if len(segment) >= min_text_length:
                            qa_pair = generate_qa_pairs(
                                client, segment, filename, 
                                page['page_marker'], few_shot_examples
                            )
                            if qa_pair:  # 只有生成成功才计数
                                all_qa_pairs.append(qa_pair)
                                doc_qa_count += 1
                                pbar.set_postfix_str(f"当前文件: {filename[:15]}..., 已生成: {doc_qa_count}/{qa_per_doc}")
                
                pbar.set_postfix_str(f"完成: {filename[:15]}..., 生成 {doc_qa_count} QA对")
            except Exception as e:
                print(f"\n处理文件 {filename} 时出错: {e}")
                continue
    
    # 保存结果
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2, 
                 separators=(',', ': '), sort_keys=True)
    
    print(f"\n处理完成！共处理 {total_files} 个文档，生成 {len(all_qa_pairs)} 个QA对")
    print(f"结果已保存到: {os.path.abspath(output_path)}")

# 示例few-shot prompt
FEW_SHOT_EXAMPLES = """
{
    "examples": [
        {
            "question": "投资相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）首年寬限期有多久？若寬限期過後仍未繳交保費會怎樣？",
            "answer": "靈活投資寶╱萬利保障計劃╱宏寶保單將享有 30 日的寬限期，宏利精選投資保之寬限期為 31 日。若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。",
            "文档来源": "电子行政运作手册_保单行政_2023_10.md",
            "页码": "6",
            "文本片段": "投資相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）\n\na) 寬限期\n\n假如首年內的逾期最低保費（MPRE/Contract Premium）或基金帳戶淨值不足以支付保單月費，客戶會收到保費備忘書。  \n靈活投資寶╱萬利保障計劃╱宏寶保單將享有30 日的寬限期（宏利精選投資保之寬限期為31 日）。  \n若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。"
        },
        {
            "question": "投資相連壽險計劃保單（邁駿投資理財計劃）在第二年淨保費測試（第十三、十四、十五個月結日），什麼情況下保險顧問的營業積分將被扣回？",
            "answer": "在第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 ≤（N - 2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。",
            "文档来源": "电子行政运作手册_保单行政_2023_10.md",
            "页码": "7",
            "文本片段": "淨保費測試（Net Premium Test）：\n\n淨保費測試決定自保單生效日以後的淨已繳保費是否足夠支付到期定期總保費。  \n淨已繳保費 $\\mathbf{\\Sigma}=\\mathbf{\\Sigma}$ 已繳至基本儲蓄帳戶之保費總額（包括「自動繳付保費」之金額，如適用）–每次從基本儲蓄帳戶提取時 (如適用)超出當時免贖回費之基金提取限額的金額之累積總額保費備忘書（如自動轉帳拒付，則發出自動轉帳拒付通知書）將在（N）月結日的淨已繳保費 $\\leqq$ （N–1）月結日的到期定期總保費時發出。  \n另一份保費備忘書將於其後的一個月結日仍未收到所需的付款時發出。若連續三個月結日仍未收到所需的付款╱所需的付款不足以支付到期定期總保費及保單未能執行「自動繳付保費」時，保單月費中的欠繳保費手續費將從基本儲蓄帳戶內扣除。  \n於第一年淨保費測試（第十二、十三、十四個月結日）及第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 $\\leqq$ （N–2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。"
        }
    ]
}
"""

if __name__ == "__main__":
    input_folder = input("请输入文件夹路径: ").strip().strip('"').strip("'")
    output_path = input("请输入输出文件路径: ").strip().strip('"').strip("'")
    qa_per_doc = int(input("请输入每个文档最多生成的QA对数量(默认3): ") or 3)
    
    if not os.path.isdir(input_folder):
        print(f"错误: 文件夹路径不存在 - {input_folder}")
        exit(1)

    process_documents(
        input_folder=input_folder,
        output_path=output_path,
        few_shot_examples=FEW_SHOT_EXAMPLES,
        qa_per_doc=qa_per_doc
    )


开始处理，共发现 1 个文档


处理进度:   0%|          | 0/1 [00:00<?, ?文件/s, 当前文件: 宏利環球貨幣保障計劃 保單條款...]


生成QA对时出错: Expecting value: line 1 column 1 (char 0)

生成QA对时出错: Extra data: line 9 column 1 (char 206)

生成QA对时出错: Extra data: line 7 column 2 (char 170)

生成QA对时出错: Expecting value: line 1 column 1 (char 0)

生成QA对时出错: Extra data: line 7 column 2 (char 182)

生成QA对时出错: Extra data: line 9 column 1 (char 179)

生成QA对时出错: Extra data: line 9 column 1 (char 229)

生成QA对时出错: Extra data: line 9 column 1 (char 236)


处理进度:   0%|          | 0/1 [01:50<?, ?文件/s, 当前文件: 宏利環球貨幣保障計劃 保單條款..., 已生成: 1/10]


生成QA对时出错: Extra data: line 8 column 1 (char 207)

生成QA对时出错: Extra data: line 9 column 1 (char 168)

生成QA对时出错: Extra data: line 7 column 2 (char 200)

生成QA对时出错: Extra data: line 7 column 2 (char 193)

生成QA对时出错: Extra data: line 7 column 2 (char 201)

生成QA对时出错: Extra data: line 7 column 2 (char 245)

生成QA对时出错: Extra data: line 8 column 1 (char 149)

生成QA对时出错: Extra data: line 7 column 2 (char 172)

生成QA对时出错: Extra data: line 7 column 2 (char 163)

生成QA对时出错: Expecting value: line 1 column 1 (char 0)

生成QA对时出错: Extra data: line 7 column 2 (char 190)

生成QA对时出错: Extra data: line 7 column 2 (char 249)

生成QA对时出错: Extra data: line 9 column 1 (char 189)


处理进度:   0%|          | 0/1 [05:27<?, ?文件/s, 当前文件: 宏利環球貨幣保障計劃 保單條款..., 已生成: 2/10]


生成QA对时出错: Extra data: line 9 column 1 (char 554)

生成QA对时出错: Extra data: line 7 column 2 (char 164)


处理进度: 100%|██████████| 1/1 [06:29<00:00, 389.36s/文件, 完成: 宏利環球貨幣保障計劃 保單條款..., 生成 2 QA对]


生成QA对时出错: Extra data: line 7 column 2 (char 163)

处理完成！共处理 1 个文档，生成 2 个QA对
结果已保存到: D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\文档1QA对


代码运行时间过长的原因:
1.API调用延迟:每次调用GLM-4 API需要约2-5秒响应时间,21个QA对意味着至少21次API调用，仅网络请求就需要约1.5-3分钟,200个则需要12小时
2.文本处理开销:文件读取、分页、分段等本地操作消耗时间
3.串行处理模式:当前代码是单文件→单页面→单片段→单API调用的串行流程

提高速度(异步并发API调用)

In [ ]:
import os
import json
import re
import asyncio
import aiohttp
from tqdm import tqdm
from zhipuai import AsyncZhipuAI

# 配置参数
API_KEY = "2a2299dd95944956b69397a89113d5a7.GW5jR7NYhANOuGES" 
MAX_CONCURRENT_REQUESTS = 5  
TIMEOUT = 15  
MIN_TEXT_LENGTH = 100  

def extract_text_from_file(file_path):
    """从文件中提取文本内容，包括分页信息（保持你原有实现）"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    pages = re.split(r'(第\d+页|\n\s*Page \d+\s*\n)', content)
    structured_pages = []
    
    for i in range(1, len(pages), 2):
        page_marker = pages[i].strip()
        page_content = pages[i+1].strip() if i+1 < len(pages) else ""
        if page_content:
            structured_pages.append({
                "page_marker": page_marker,
                "content": page_content
            })
    return structured_pages

def split_text_into_segments(text, max_length=600):
    """将文本分割成有意义的片段（保持你原有实现）"""
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
    segments = []
    
    for para in paragraphs:
        if len(para) <= max_length:
            segments.append(para)
        else:
            sentences = re.split(r'(?<=[。！？；;])\s+', para)
            current_segment = ""
            for sent in sentences:
                if len(current_segment) + len(sent) <= max_length:
                    current_segment += sent + " "
                else:
                    if current_segment:
                        segments.append(current_segment.strip())
                    current_segment = sent + " "
            if current_segment:
                segments.append(current_segment.strip())
    return segments

def is_valid_content(text):
    """跳过目录/页码等非正文内容（新增函数）"""
    text = text.strip()
    # 跳过条件
    skip_patterns = [
        r'\.{3,}\d+$',          
        r'第[一二三四五六七八九十]+页', 
        r'^[©®]|保密$'         
    ]
    if any(re.search(p, text) for p in skip_patterns):
        return False
    # 有效性检查
    has_chinese = re.search(r'[\u4e00-\u9fa5]', text)
    valid_length = len(text) >= MIN_TEXT_LENGTH
    return has_chinese and valid_length

class QAGenerator:
    def __init__(self):
        self.client = AsyncZhipuAI(api_key=API_KEY)
        self.few_shot_examples = """
        {
            "examples": [
                {
                    "question": "投资相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）首年寬限期有多久？若寬限期過後仍未繳交保費會怎樣？",
                    "answer": "靈活投資寶╱萬利保障計劃╱宏寶保單將享有 30 日的寬限期，宏利精選投資保之寬限期為 31 日。若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。",
                    "source_file": "电子行政运作手册_保单行政_2023_10.md",
                    "page_num": "6",
                    "content": "投資相連壽險計劃保單（萬利保障計劃、宏寶、靈活投資寶及宏利精選投資保）\n\na) 寬限期\n\n假如首年內的逾期最低保費（MPRE/Contract Premium）或基金帳戶淨值不足以支付保單月費，客戶會收到保費備忘書。  \n靈活投資寶╱萬利保障計劃╱宏寶保單將享有30 日的寬限期（宏利精選投資保之寬限期為31 日）。  \n若寬限期過後仍未繳交保費，客戶會收到保單失效通知書，內附有關保單復效的規定。"
                },
                {
                    "question": "投資相連壽險計劃保單（邁駿投資理財計劃）在第二年淨保費測試（第十三、十四、十五個月結日），什麼情況下保險顧問的營業積分將被扣回？",
                    "answer": "在第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 ≤（N - 2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。",
                    "source_file": "电子行政运作手册_保单行政_2023_10.md",
                    "page_num": "7",
                    "content": "淨保費測試（Net Premium Test）：\n\n淨保費測試決定自保單生效日以後的淨已繳保費是否足夠支付到期定期總保費。  \n淨已繳保費 $\\mathbf{\\Sigma}=\\mathbf{\\Sigma}$ 已繳至基本儲蓄帳戶之保費總額（包括「自動繳付保費」之金額，如適用）–每次從基本儲蓄帳戶提取時 (如適用)超出當時免贖回費之基金提取限額的金額之累積總額保費備忘書（如自動轉帳拒付，則發出自動轉帳拒付通知書）將在（N）月結日的淨已繳保費 $\\leqq$ （N–1）月結日的到期定期總保費時發出。  \n另一份保費備忘書將於其後的一個月結日仍未收到所需的付款時發出。若連續三個月結日仍未收到所需的付款╱所需的付款不足以支付到期定期總保費及保單未能執行「自動繳付保費」時，保單月費中的欠繳保費手續費將從基本儲蓄帳戶內扣除。  \n於第一年淨保費測試（第十二、十三、十四個月結日）及第二年淨保費測試（第十三、十四、十五個月結日），若（N）月結日的淨已繳保費 $\\leqq$ （N–2）月結日的到期定期總保費時，保險顧問的營業積分將被扣回。"
                }
                        ]
        }
        """

    async def generate_qa(self, text_segment: str, doc_name: str, page_info: str):
        """异步生成QA对"""
        prompt = f"""请严格按以下要求生成QA对：
1. 问题必须针对文本中的具体条款
2. 答案必须直接引用原文片段
3. 使用此JSON格式：
{{
  "question": "问题",
  "answer": "答案",
  "content": "引用的原文",
  "source_file": "{doc_name}",
  "page_num": "{page_info}"
}}

待分析文本：
{text_segment}"""
        
        try:
            response = await self.client.chat.completions.create(
                model="glm-4",
                messages=[
                    {"role": "system", "content": "你是一个专业的文档分析助手"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.2,
                max_tokens=800,
                timeout=TIMEOUT
            )
            
            qa_pair = json.loads(response.choices[0].message.content)
            return {
                **qa_pair,
                "source_file": doc_name,
                "page_num": page_info,
                "content": text_segment[:300] + "..." if len(text_segment) > 300 else text_segment
            }
        except Exception as e:
            tqdm.write(f"生成QA对时出错: {str(e)[:100]}...")
            return None

async def async_process_documents(input_folder: str, output_path: str, qa_per_doc: int = 5):
    """异步处理文档主函数"""
    generator = QAGenerator()
    all_qa_pairs = []
    file_list = [f for f in os.listdir(input_folder) if f.endswith(('.md', '.txt', '.pdf'))]
    
    with tqdm(total=len(file_list), desc="文档处理进度") as pbar:
        for filename in file_list:
            try:
                file_path = os.path.join(input_folder, filename)
                pages = extract_text_from_file(file_path)
                doc_qa_pairs = []
                
                for page in pages:
                    segments = split_text_into_segments(page['content'])
                    valid_segments = [s for s in segments if is_valid_content(s)][:qa_per_doc]
                    
                    tasks = [generator.generate_qa(s, filename, page['page_marker']) for s in valid_segments]
                    results = await asyncio.gather(*tasks)
                    
                    doc_qa_pairs.extend([r for r in results if r is not None])
                    if len(doc_qa_pairs) >= qa_per_doc:
                        break
                
                all_qa_pairs.extend(doc_qa_pairs[:qa_per_doc])
                pbar.update(1)
                pbar.set_postfix_str(f"生成: {len(doc_qa_pairs)}/{qa_per_doc}")
                
            except Exception as e:
                tqdm.write(f"处理文件 {filename} 出错: {e}")
                continue
    
    # 保存结果
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(all_qa_pairs, f, ensure_ascii=False, indent=2)
    
    print(f"\n处理完成！共生成 {len(all_qa_pairs)} 个QA对")
    print(f"结果已保存到: {os.path.abspath(output_path)}")

if __name__ == "__main__":
    input_folder = input("请输入文件夹路径: ").strip('"\'')
    output_path = input("请输入输出文件路径: ").strip('"\'')
    qa_per_doc = int(input("每个文档生成QA数(默认30): ") or 30)
    
    if not os.path.isdir(input_folder):
        print(f"错误: 文件夹路径不存在 - {input_folder}")
        exit(1)
    
    asyncio.run(async_process_documents(input_folder, output_path, qa_per_doc))

ImportError: cannot import name 'AsyncZhipuAI' from 'zhipuai' (d:\anaconda\Lib\site-packages\zhipuai\__init__.py)

智谱AI的官方Python SDK 可能尚未正式发布异步版本，导致无法导入 AsyncZhipuAI
QA对质量:存在没意义的问题;没能平均从文档各页中提取QA对,而是逐页提取,满足数量就读取下一文档;回答不完整,只是简单对应文本片段